### 글로벌 도시 현재시간 & 날씨 crawling

In [ ]:
import time
import csv
import pandas as pd
import schedule

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from textrankr import TextRank
from typing import List
from newspaper import Article

In [ ]:
# driver 설정
from selenium import webdriver 
from selenium.webdriver.common.by import By
        
# chrome driver
driver = webdriver.Chrome()

### IT뉴스(오전 9:30수집)

In [ ]:
def crawling_news():
    
    # 리스트 형식으로 저장하기 위한 클래스 선언
    class MyTokenizer:
        def __call__(self,text:str)->List[str]:
            tokens: List[str]=text.split()
            return tokens
    # 리스트 형식으로 저장하는 공간만들고 그 공간에 데이터 요약
    mytokenizer: MyTokenizer=MyTokenizer()
    textrank: TextRank = TextRank(mytokenizer)
    k=3 # 요약 데이터 3줄까지 설정
    
    
    url='https://news.naver.com/section/105'
    driver.get(url)
    time.sleep(0.3)
    
    driver.find_element(By.XPATH,'//*[@id="newsct"]/div[1]/div[2]/a').click()
    time.sleep(0.3)
    
    # 뉴스 기사가 몇개 잇는지
    news_list=driver.find_elements(By.CLASS_NAME,'sa_item._SECTION_HEADLINE')

    # csv저장을 위한 dataframe 생성
    df = pd.DataFrame(columns=['title','company','url','summarize_content','content','ai_summary'])

    title_list=[]
    company_list=[]
    summarize_content_list=[]
    url_list=[]
    content_list=[]
    ai_summary_content_list=[]

    for news in news_list:
        print('-----------')
        title = news.find_element(By.CLASS_NAME, 'sa_text_strong').text #뉴스제목
        company = news.find_element(By.CLASS_NAME, 'sa_text_press').text #언론사
        summarize_content = news.find_element(By.CLASS_NAME, 'sa_text_lede').text #내용
        url = news.find_element(By.CLASS_NAME, 'sa_text_title').get_attribute('href') #작성일

        title_list.append(title)
        company_list.append(company)
        summarize_content_list.append(summarize_content)
        url_list.append(url)

    for url in url_list:
        driver.get(url)
        content=driver.find_element(By.CLASS_NAME, 'go_trans._article_content').text #내용
        print(content)
        ai_summary: str = textrank.summarize(content,k)
        print(ai_summary) 
        
        time.sleep(1)
        content_list.append(content)
        ai_summary_content_list.append(ai_summary)
    
    # df.loc[cnt] = [title,company,summarize_content,ai_summary,url]
    df['title']=title_list
    df['company']=company_list
    df['url']=url_list
    df['summarize_content']=summarize_content_list
    df['content']=content_list
    df['ai_summary']=ai_summary_content_list
    
    # dataframe를 csv로 저장
    df.to_csv("news.csv", encoding='utf-8-sig',index=False)

In [ ]:
# schedule.every().day.at("09:30").do(crawling_news)
schedule.every(14).seconds.do(crawling_news)
while True:
    schedule.run_pending()
    time.sleep(1)

### 웹사이트에서 도시 검색

뉴욕 - 런던 - 파리 - 도쿄 - 베이징 - 홍콩 - 로스앤젤레스 - 시카고 - 싱가포르 - 워싱턴 D.C.

In [ ]:
url='https://m.naver.com/'
driver.get(url)
time.sleep(1)

# 세계 도시 목록 
global_citys=['뉴욕','런던','파리','도쿄','베이징','홍콩','로스앤젤레스','시카고','싱가포르','워싱턴 D.C.']

# 사용자의 응답 중 검색원하는 도시 추출
citys=[]
for user_answer in user_answers:
    if user_answer in global_citys:
        citys.append(user_answer)

# 일단은 첫번째 도시만 검색하게 구현
city=citys[0]

# 사용자가 무엇을 원하는지? 1) 날씨 2) 현지시각 3)뉴스
selection=['날씨','현지시각']

driver.find_element(By.XPATH,'//*[@id="MM_SEARCH_FAKE"]').click()
driver.find_element(By.XPATH,'//*[@id="query"]').send_keys(f'{city} 날씨')
driver.find_element(By.XPATH,'//*[@id="sch_w"]/div/form/button').click()
time.sleep(1)

# 글로벌 도시 날씨
if '날씨' in selection:
    
    # csv저장을 위한 dataframe 생성
    df = pd.DataFrame(columns=['city','temperature','sky','feel_temperature']) 
    
    weather = driver.find_element(By.XPATH,'//*[@id="ct"]/section[1]/div/div[2]/div/div/div[1]/div[1]/div/div[2]/div/div[1]/div[2]').text
    weather=weather.split('\n') 
    temperature=" ".join(weather[0:2])
    sky=weather[2]
    feel_temperature=weather[3]
    
    print(temperature, sky, feel_temperature)
    df.loc[0] = [city,temperature, sky, feel_temperature]
    
    # dataframe를 csv로 저장
    df.to_csv("city_weather.csv", encoding='utf-8-sig',index=False) # dataframe을 csv로

# 글로벌 도시 현지시각    
if '현지시각' in selection:
    
    # csv저장을 위한 dataframe 생성
    df = pd.DataFrame(columns=['city','city_time']) 
    
    city_time=driver.find_element(By.XPATH,'//*[@id="ct"]/section[1]/div/div[2]/div/div/div[1]/div[1]/div/div[2]/div/div[2]/dl[1]/dd[1]').text
    print(city_time)
    
    df.loc[0] = [city,city_time]

    # dataframe를 csv로 저장
    df.to_csv("city_time.csv", encoding='utf-8-sig',index=False) # dataframe을 csv로

# 글로벌 뉴스 
if '뉴스' in selection :
  # csv 만들고 보내는 코드 추후 구현
  pass

# 뉴스 데이터분석

### 뉴스 구성 단어 분석

In [32]:
from konlpy.tag import Okt
import pandas as pd
import numpy as np

In [12]:
okt = Okt()

df=pd.read_csv('news.csv',encoding='utf-8-sig')
df['content_token'] = df['content'].apply(lambda x: ','.join([word for word, pos in okt.pos(x) if pos in ['Noun']])) #'Adjective','Verb'

In [34]:
df[['content_token']]

,content_token
0,"책임감,기술,서비스,개발,집중,신뢰,안전,생태계,구축,협력,카카오,글로벌,오픈,소스..."
1,"과학기술,정보통신부,한국,지능,정보사회,진흥,데이터,플래그,사업,과,데이터,활용,기..."
2,"막스,플랑크,연구,회,글로벌,과학,리더,포럼,공동,개최,과,정통부,제공,서울,뉴스,..."
3,"달,개최,화성,탐사,체험,등,계,프로그램,마련,국립과천과학관,제공,서울,뉴스,윤주영..."
4,"이동수,네이버,클라우드,이사,인텔,칩,성능,가격,매력,엔비디아,독주,칩,시장,인텔,..."
5,"국내,게임,최초,게임,어워즈,수상,넥슨,제공,뉴스,서울,뉴스,김승준,기자,넥슨,영국..."
6,"미생물,식품,생산,활용,여러,지속,원료,및,활용,미생물,바이오매스,생산,전략,제공,..."
7,"어도비,마이크로소프트,파트너,체결,발표,새,파트너,양사,어도비,익스,피리,언스,클라..."


### WordCloud

In [ ]:
from wordcloud import WordCloud

In [38]:
news_words=np.concatenate(df['content_token'].str.split(',').apply(np.array).tolist())

In [39]:
len(news_words)

1727

In [40]:
from collections import Counter
counts = Counter(news_words)
print(counts)

Counter({'등': 32, '수': 26, '및': 22, '미생물': 22, '인텔': 21, '데이터': 21, '마이크로소프트': 17, '식품': 17, '네이버': 16, '어도비': 15, '활용': 14, '생산': 14, '협력': 13, '클라우드': 12, '것': 11, '통해': 11, '사업': 11, '연구': 10, '를': 10, '가우디': 10, '지원': 9, '고': 9, '더': 9, '게임': 9, '마케터': 9, '서비스': 8, '개발': 8, '글로벌': 8, '위해': 8, '지속': 8, '제공': 8, '업무': 8, '인사이트': 8, '애플리케이션': 8, '캠페인': 8, '지능': 7, '한국': 7, '기획': 7, '기반': 7, '공동': 7, '마케팅': 7, '카카오': 6, '국내': 6, '인공': 6, '여러': 6, '말': 6, '과': 6, '검증': 6, '과제': 6, '서울': 6, '뉴스': 6, '센터': 6, '이': 6, '칩': 6, '엔비디아': 6, '시장': 6, '원료': 6, '바이오매스': 6, '각종': 6, '단백질': 6, '용': 6, '코': 6, '파일럿': 6, '기술': 5, '얼라이언스': 5, '가입': 5, '기자': 5, '기업': 5, '분야': 5, '의': 5, '데': 5, '프로젝트': 5, '리더': 5, '사회': 5, '플래그': 5, '해결': 5, '확인': 5, '효율': 5, '기초과학': 5, '양': 5, '역량': 5, '과정': 5, '이동수': 5, '이사': 5, '며': 5, '데이브': 5, '교수': 5, '익스': 5, '피리': 5, '언스': 5, '안전': 4, '생태계': 4, '구축': 4, '오픈': 4, '혁신': 4, '목표': 4, '계획': 4, '교육': 4, '이번': 4, '또': 4, '예정': 4, '디지털': 4, '과학기술': 4, '개': 4, '정통부': 4,

### 도시이름 잘 검색되는지 한번에 확인하고 싶을때 부록

In [ ]:
# 한번에 확인하고 싶을때
citys=['뉴욕','런던','파리','도쿄','베이징','홍콩','로스앤젤레스','시카고','싱가포르','워싱턴DC']
for city in citys:
    driver.get(url)
    time.sleep(1)
    driver.find_element(By.XPATH,'//*[@id="MM_SEARCH_FAKE"]').click()
    driver.find_element(By.XPATH,'//*[@id="query"]').send_keys(f'{city} 날씨')
    driver.find_element(By.XPATH,'//*[@id="sch_w"]/div/form/button').click()
    time.sleep(1)
    print(city)
    
    # 글로벌 도시 날씨
    weather = driver.find_element(By.XPATH,'//*[@id="ct"]/section[1]/div/div[2]/div/div/div[1]/div[1]/div/div[2]/div/div[1]/div[2]').text
    weather=weather.split('\n') 
    temperature=" ".join(weather[0:2])
    sky=weather[2]
    feel_temperature=weather[3]
    print(temperature, sky, feel_temperature)

    # 글로벌 도시 현지시각
    city_time=driver.find_element(By.XPATH,'//*[@id="ct"]/section[1]/div/div[2]/div/div/div[1]/div[1]/div/div[2]/div/div[2]/dl[1]/dd[1]').text
    print(city_time)